In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigscience/bloomz-560m"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype="auto", device_map="auto")


c:\hammad workings\Thesis\Multihop for Urdu\Multihop for Urdu\code\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\hammad workings\Thesis\Multihop for Urdu\Multihop for Urdu\code\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\23030013\.cache\huggingface\hub\models--bigscience--bloomz-560m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate De

Translate to English: Je t’aime. I love you.


In [ ]:
# save_directory = "C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\model_weights\\bloomz-560m"
# model.save_pretrained(save_directory)
# tokenizer.save_pretrained(save_directory)

('C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\model_weights\\bloomz-560m\\tokenizer_config.json',
 'C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\model_weights\\bloomz-560m\\special_tokens_map.json',
 'C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\model_weights\\bloomz-560m\\tokenizer.json')

In [5]:
load_directory = "C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\model_weights\\bloomz-560m"
tokenizer = AutoTokenizer.from_pretrained(load_directory)
model = AutoModelForCausalLM.from_pretrained(load_directory, torch_dtype="auto", device_map="auto")

In [9]:
def classify_urdu_question(urdu_question):
    # Create the prompt with English instructions and Urdu question
    prompt = f"""Classify the following Urdu question as either "simple" (requires no context), "singlehop" (requires one inference step), or "multihop" (requires multiple reasoning steps):

{urdu_question}

Classification:"""
    
    # Tokenize and generate
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        inputs, 
        max_new_tokens=10,  # We only need a short answer
        temperature=0.1,    # Lower temperature for more deterministic output
        do_sample=False,    # Use greedy decoding for consistent results
        num_beams=3         # Simple beam search for better quality
    )
    
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract just the classification from the result
    # This will return everything after the prompt
    classification = result.replace(prompt, "").strip()
    
    return classification

In [17]:
def classify_urdu_question(urdu_question):
    prompt = f"""
You are a reasoning expert. Your task is to classify the complexity of a question written in Urdu into one of three categories:

- "simple": A fact-based question that can be answered without needing additional context or inference.
- "singlehop": A question that requires one reasoning step or fact connection.
- "multihop": A question that needs multiple reasoning steps or combining information from different parts.

Here are a few examples:

Example 1:
Question (Urdu): پاکستان کا دارالحکومت کیا ہے؟
Classification: simple

Example 2:
Question (Urdu): وہ شخص کون تھا جو قائد اعظم کے بعد گورنر جنرل بنا؟
Classification: singlehop

Example 3:
Question (Urdu): وہ سائنسدان کون ہے جس نے نظریہ ارتقاء پیش کیا، اور اس کا اثر بیسویں صدی کی حیاتیات پر کیا پڑا؟
Classification: multihop

Now classify the following question:

Question (Urdu): {urdu_question}
Classification:"""

    inputs = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        inputs,
        max_new_tokens=10,
        temperature=0.1,
        do_sample=False,
        num_beams=3
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    classification = result.replace(prompt, "").strip()

    return classification


In [18]:
# Example usage
urdu_question = "پاکستان کا دارالحکومت کیا ہے؟"  # "What is the capital of Pakistan?"
classification = classify_urdu_question(urdu_question)
print(f"Question: {urdu_question}")
print(f"Classification: {classification}")

# You can test with different complexity questions
urdu_question2 = "جس ملک کی سرحد چین کے ساتھ ملتی ہے اس کا دارالحکومت کیا ہے؟"  # Multihop: "What is the capital of the country that borders China?"
classification2 = classify_urdu_question(urdu_question2)
print(f"Question: {urdu_question2}")
print(f"Classification: {classification2}")

c:\hammad workings\Thesis\Multihop for Urdu\Multihop for Urdu\code\venv\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Question: پاکستان کا دارالحکومت کیا ہے؟
Classification: simple
Question: جس ملک کی سرحد چین کے ساتھ ملتی ہے اس کا دارالحکومت کیا ہے؟
Classification: singlehop


In [12]:
import pandas as pd
from tqdm import tqdm  # For progress bar

# Load your CSV file
df = pd.read_csv("C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\Dataset\\Hotpotqa\\1000_paras_100_queries\\translated_dataset_100_qna.csv")  # Replace with your actual file name

# Ensure 'translated_question' column exists
if 'translated_question' not in df.columns:
    raise ValueError("Column 'translated_question' not found in the CSV.")

# Add a new column for the classification results
tqdm.pandas()  # Enables progress bar with apply
df['question_type'] = df['translated_question'].progress_apply(classify_urdu_question)


  0%|          | 0/98 [00:00<?, ?it/s]

c:\hammad workings\Thesis\Multihop for Urdu\Multihop for Urdu\code\venv\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 98/98 [13:57<00:00,  8.55s/it]


In [90]:
import ollama

def classify_urdu_question(urdu_question):
    prompt = f"""
You are a reasoning expert. Your task is to classify the complexity of a question written in Urdu into one of three categories:

- "simple": A fact-based question that can be answered without needing additional context or inference.
- "singlehop": A question that requires one reasoning step or fact connection.
- "multihop": A question that needs multiple reasoning steps or combining information from different parts.

Here are a few examples:

Example 1:
Question (Urdu): پاکستان کا دارالحکومت کیا ہے؟
Classification: simple

Example 2:
Question (Urdu): وہ شخص کون تھا جو قائد اعظم کے بعد گورنر جنرل بنا؟
Classification: singlehop

Example 3:
Question (Urdu): وہ سائنسدان کون ہے جس نے نظریہ ارتقاء پیش کیا، اور اس کا اثر بیسویں صدی کی حیاتیات پر کیا پڑا؟
Classification: multihop

Now classify the following question:

Question (Urdu): {urdu_question}
Classification:"""

    try:
        response = ollama.chat(
            model='llama3:8b',
            messages=[{"role": "user", "content": prompt}]
        )
        reply = response['message']['content'].strip().lower()

        # Normalize to allowed values
        for label in ['simple', 'singlehop', 'multihop']:
            if label in reply:
                return label

        print(f"⚠️ Unexpected response: {reply}")
        return "unknown"

    except Exception as e:
        print(f"❌ Error processing question: {urdu_question}\n↪ {e}")
        return "error"



import pandas as pd
from tqdm import tqdm

# Load your CSV
df = pd.read_csv("C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\Dataset\\Hotpotqa\\1000_paras_100_queries\\translated_dataset_100_qna.csv")

# Make sure necessary columns exist
required_columns = {'translated_question', 'level'}
if not required_columns.issubset(df.columns):
    raise ValueError(f"Missing one of the required columns: {required_columns - set(df.columns)}")

# Enable tqdm for apply
tqdm.pandas()

# Generate temporary 'question_type' in memory (NOT saving to CSV)
df_temp = df.copy()
df_temp['question_type'] = df_temp['translated_question'].progress_apply(classify_urdu_question)

# Define classification check logic
def is_correct(row):
    level = str(row['level']).strip().lower()
    q_type = str(row['question_type']).strip().lower()

    if level == 'easy' and q_type in {'simple', 'singlehop'}:
        return True
    elif level in {'medium', 'hard'} and q_type == 'multihop':
        return True
    else:
        return False

# Apply correctness check
df_temp['correct_classification'] = df_temp.apply(is_correct, axis=1)

# Summary stats
total = len(df_temp)
correct = df_temp['correct_classification'].sum()
accuracy = correct / total * 100

print(f"\n✅ Total questions: {total}")
print(f"✅ Correctly classified: {correct}")
print(f"✅ Accuracy: {accuracy:.2f}%")

# Per-level analysis
print("\n🔍 Accuracy by level:")
for level in df_temp['level'].unique():
    subset = df_temp[df_temp['level'].str.lower() == level.lower()]
    correct_subset = subset['correct_classification'].sum()
    total_subset = len(subset)
    acc = correct_subset / total_subset * 100 if total_subset else 0
    print(f"  {level}: {acc:.2f}% ({correct_subset}/{total_subset})")


  0%|          | 0/98 [00:00<?, ?it/s]

100%|██████████| 98/98 [04:09<00:00,  2.55s/it]


✅ Total questions: 98
✅ Correctly classified: 39
✅ Accuracy: 39.80%

🔍 Accuracy by level:
  easy: 44.44% (8/18)
  medium: 46.67% (14/30)
  hard: 34.00% (17/50)


In [1]:
import ollama
ollama.pull("llama3:8b")

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [7]:
import ollama
import pandas as pd
from tqdm import tqdm

# --- Classification Function ---
def classify_urdu_question(urdu_question):
    prompt = f"""
You are a reasoning expert. Your task is to classify the complexity of a question written in Urdu into one of two categories:

- "singlehop": A question that can be answered with one reasoning step or simple fact retrieval.
- "multihop": A question that requires multiple reasoning steps or combining information from different parts.

Here are a few examples:

Example 1:
Question (Urdu): پاکستان کا دارالحکومت کیا ہے؟
Classification: singlehop

Example 2:
Question (Urdu): وہ شخص کون تھا جو قائد اعظم کے بعد گورنر جنرل بنا؟
Classification: singlehop

Example 3:
Question (Urdu): وہ سائنسدان کون ہے جس نے نظریہ ارتقاء پیش کیا، اور اس کا اثر بیسویں صدی کی حیاتیات پر کیا پڑا؟
Classification: multihop

Now classify the following question:

Question (Urdu): {urdu_question}
Classification:"""

    try:
        response = ollama.chat(
            model='llama3:8b',
            messages=[{"role": "user", "content": prompt}]
        )
        reply = response['message']['content'].strip().lower()

        # Normalize output
        if 'multihop' in reply:
            return 'multihop'
        elif 'singlehop' in reply or 'simple' in reply:
            return 'singlehop'

        print(f"⚠️ Unexpected response: {reply}")
        return "unknown"

    except Exception as e:
        print(f"❌ Error processing question: {urdu_question}\n↪ {e}")
        return "error"

# --- Load Dataset ---
df = pd.read_csv("C:\\hammad workings\\Thesis\\Multihop for Urdu\\Multihop for Urdu\\Dataset\\Hotpotqa\\1000_paras_100_queries\\translated_dataset_100_qna.csv")

# Ensure required columns exist
required_columns = {'translated_question', 'level'}
if not required_columns.issubset(df.columns):
    raise ValueError(f"Missing one of the required columns: {required_columns - set(df.columns)}")

# Enable progress bar for apply
tqdm.pandas()

# Create temporary working DataFrame
df_temp = df.copy()
df_temp['question_type'] = df_temp['translated_question'].progress_apply(classify_urdu_question)

# --- Evaluation Logic ---
def is_correct(row):
    level = str(row['level']).strip().lower()
    q_type = str(row['question_type']).strip().lower()

    if level == 'easy' and q_type == 'singlehop':
        return True
    elif level in {'medium', 'hard'} and q_type == 'multihop':
        return True
    else:
        return False

df_temp['correct_classification'] = df_temp.apply(is_correct, axis=1)

# --- Evaluation Summary ---
total = len(df_temp)
correct = df_temp['correct_classification'].sum()
accuracy = correct / total * 100

print(f"\n✅ Total questions: {total}")
print(f"✅ Correctly classified: {correct}")
print(f"✅ Accuracy: {accuracy:.2f}%")

# --- Per-level Accuracy ---
print("\n🔍 Accuracy by level:")
for level in df_temp['level'].unique():
    subset = df_temp[df_temp['level'].str.lower() == level.lower()]
    correct_subset = subset['correct_classification'].sum()
    total_subset = len(subset)
    acc = correct_subset / total_subset * 100 if total_subset else 0
    print(f"  {level}: {acc:.2f}% ({correct_subset}/{total_subset})")

# --- Optional: Display misclassified examples ---
print("\n❌ Misclassified questions:")
print(df_temp[df_temp['correct_classification'] == False][['translated_question', 'level', 'question_type']].head(10))


  0%|          | 0/98 [00:00<?, ?it/s]

100%|██████████| 98/98 [04:35<00:00,  2.81s/it]


✅ Total questions: 98
✅ Correctly classified: 64
✅ Accuracy: 65.31%

🔍 Accuracy by level:
  easy: 16.67% (3/18)
  medium: 73.33% (22/30)
  hard: 78.00% (39/50)

❌ Misclassified questions:
                                  translated_question level question_type
0   سیری بی 2017 - 2017 (اسپانسرشپ کی وجوہات کی بن...  easy      multihop
1   "آکسفورڈ کالج کے ایک ساتھی ایلک نیلر ڈکن نے کہ...  easy      multihop
2   "جراسک پارک کے اداکار ڈیوڈ ہنری ہوانگ نے ""دی ...  easy      multihop
3   کون سا کردار ، ڈین کاسٹیلینیٹا کی آواز ، سمپسن...  easy      multihop
5   امریکی باسکٹ بال ٹیم نے اس سال کے آخر میں شکاگ...  easy      multihop
6   لیٹروب بریوری کمپنی ، جو 1839 میں قائم ہوئی تھ...  easy      multihop
7   امریکی فوج کے ایک افسر اور منشیات کے اسمگلنگ ک...  easy      multihop
8   اینڈریو مارٹن ڈوبر ، ایک امریکی مخلوط مارشل آر...  easy      multihop
9   ایمینم کے البم مارشل میتھرز ایل پی 2 پر کون سا...  easy      multihop
11  ڈیان ولکنز اور اسٹیون اسپیلبرگ نیو ہالی ووڈ کے...  easy      multih

In [4]:
print(df_temp[df_temp['correct_classification'] == False][['translated_question', 'level', 'question_type']])


                                  translated_question   level question_type
0   سیری بی 2017 - 2017 (اسپانسرشپ کی وجوہات کی بن...    easy      multihop
1   "آکسفورڈ کالج کے ایک ساتھی ایلک نیلر ڈکن نے کہ...    easy      multihop
3   کون سا کردار ، ڈین کاسٹیلینیٹا کی آواز ، سمپسن...    easy      multihop
4      کون تھا ایک حصہ S#arp، لی Ji-hye یا کرٹس رائٹ؟    easy      multihop
5   امریکی باسکٹ بال ٹیم نے اس سال کے آخر میں شکاگ...    easy      multihop
6   لیٹروب بریوری کمپنی ، جو 1839 میں قائم ہوئی تھ...    easy      multihop
7   امریکی فوج کے ایک افسر اور منشیات کے اسمگلنگ ک...    easy      multihop
8   اینڈریو مارٹن ڈوبر ، ایک امریکی مخلوط مارشل آر...    easy      multihop
9   ایمینم کے البم مارشل میتھرز ایل پی 2 پر کون سا...    easy      multihop
12  ایمی لی اور ٹی ایس ایم ، موسیقی کی کس صنف سے م...    easy      multihop
13  ریاستہائے متحدہ امریکہ کے ہوائی اڈے پر ، یونائ...    easy      multihop
14  نیکول مچل ایک امریکی موسمیاتی ماہر ہیں جو پہلی...    easy      multihop
15  "پسیوڈوٹ

In [14]:
df_temp['correct_classification']

0     False
1     False
2      True
3      True
4     False
      ...  
93    False
94    False
95    False
96     True
97    False
Name: correct_classification, Length: 98, dtype: bool

In [15]:
df_temp['question_type']

0                                             _multihop
1     Question: What is the best title for this sent...
2                                             singlehop
3                                             singlehop
4                                not enough information
                            ...                        
93                                               simple
94                                               simple
95                                            singlehop
96                                             multihop
97                                            singlehop
Name: question_type, Length: 98, dtype: object

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "bigscience/bloomz-3b"  # or bloom-1b7/bloom-3b
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

c:\hammad workings\Thesis\Multihop for Urdu\Multihop for Urdu\code\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\23030013\.cache\huggingface\hub\models--bigscience--bloomz-3b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some parameters are on the meta device because they were offloaded to the

In [27]:
input_text = """
Break the following Urdu question into 2 logical sub-questions needed to answer it:
Question: اگر لاہور میں فضائی آلودگی کی سطح زیادہ ہے اور فضائی آلودگی نمونیا کا سبب بن سکتی ہے، تو لاہور میں رہنے والوں کو نمونیا سے بچنے کے لیے کیا اقدامات کرنے چاہئیں؟
"""

inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_length=300)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Break the following Urdu question into 2 logical sub-questions needed to answer it:
Question: اگر لاہور میں فضائی آلودگی کی سطح زیادہ ہے اور فضائی آلودگی نمونیا کا سبب بن سکتی ہے، تو لاہور میں رہنے والوں کو نمونیا سے بچنے کے لیے کیا اقدامات کرنے چاہئیں؟
* لاہور میں فضائی آلودگی کی سطح زیادہ ہے


In [ ]:
import ollama
ollama.pull('mistral:latest')


ProgressResponse(status='success', completed=None, total=None, digest=None)

In [79]:
import ollama

def decompose_urdu_query(urdu_query: str) -> dict:
    """Returns dictionary with q1 and q2 keys containing sub-questions"""
    refined_prompt = f"""
**Role**: You are an expert Urdu linguistic analyst specializing in question decomposition. Your task is to break down complex Urdu questions into their fundamental components.

**Task Instructions**:
1. Carefully analyze the given Urdu question to identify its core components
2. Extract exactly 2 sub-questions that:
   - Are necessary to answer the main question
   - Cover distinct aspects of the problem
   - Have clear logical progression (answer to q1 helps answer q2)
3. Both sub-questions must:
   - Be in proper Urdu language
   - Be grammatically correct
   - Be clear and concise
   - Use relevant domain terminology

**Output Format Requirements**:
- Use EXACTLY this format:
  q1: [پہلا ذیلی سوال]
  q2: [دوسرا ذیلی سوال]
- Each sub-question must be on a new line
- Do not include any additional commentary or explanation
- Do not number the questions (use only q1:/q2: prefixes)

**Example 1**:
Input: اگر لاہور میں فضائی آلودگی کی سطح دہلی سے زیادہ ہے اور فضائی آلودگی پھیپھڑوں کے کینسر کا سبب بن سکتی ہے، تو لاہور کے رہائشیوں کو کس قسم کے طبی چیک اپ کروانے چاہئیں؟
Output:
q1: لاہور اور دہلی میں فضائی آلودگی کی سطح کا موازنہ کیا ہے؟
q2: فضائی آلودگی پھیپھڑوں کے کینسر کا سبب کیسے بنتی ہے؟

**Example 2**:
Input: اگر کراچی میں بجلی کے نرخ 30% بڑھ گئے ہیں اور یہ صنعتوں کو متاثر کر رہا ہے، تو حکومت کو کون سی سبسڈیاں دینی چاہئیں؟
Output:
q1: کراچی میں بجلی کے نرخوں میں اضافے کی موجودہ شرح کیا ہے؟
q2: بجلی کے مہنگے ہونے سے صنعتوں پر کس قسم کے اثرات مرتب ہو رہے ہیں؟

**Current Task**:
Input: {urdu_query}
Output:
"""
    
    try:
        response = ollama.generate(
            model='mistral',
            prompt=refined_prompt,
            options={
                'temperature': 0.5,
                'num_ctx': 2048
            }
        )
        
        output = response['response'].strip()
        
        result = {}
        for line in output.split('\n'):
            line = line.strip()
            if line.startswith('q1:'):
                result['q1'] = line[3:].strip()
            elif line.startswith('q2:'):
                result['q2'] = line[3:].strip()
        
        return result if len(result) == 2 else {}
    
    except Exception as e:
        print(f"Decomposition error: {str(e)}")
        return {}


In [80]:

# Correct usage:
your_query = "اگر اسلام آباد میں ٹریفک جام کی شکایات 50% بڑھ گئی ہیں اور یہ شہر کی نقل و حمل کی ناکافی پالیسیوں کی وجہ سے ہے، تو حکومت کو کون سے نئے ٹریفک قوانین متعارف کروانے چاہئیں؟"

result = decompose_urdu_query(your_query)
if result:
    print(f"q1: {result.get('q1', 'Not generated')}")
    print(f"q2: {result.get('q2', 'Not generated')}")
else:
    print("Failed to decompose the query")

q1: اسلام آباد میں ٹریفک جام کی شکایات میں اضافے کی موجودہ شرح کیا ہے؟
q2: نقل و حمل پالیسیوں کے وجہ سے ٹریفک جام کی شکایت کی بڑھنے سے کس قسم کے اثرات مرتب ہو رہے ہیں؟


In [81]:
# 1. Environmental Policy
your_query_1 = "اگر دریائے سندھ میں آلودگی کی شرح 40% بڑھ گئی ہے اور یہ زراعت کو متاثر کر رہی ہے، تو پنجاب حکومت کو کون سے فوری اقدامات کرنے چاہئیں؟"

# 2. Public Health
your_query_2 = "اگر کراچی میں ڈینگی کے کیسز میں 150% اضافہ ہوا ہے اور بارشوں کا غیر معمولی نمونہ اس کی بنیادی وجہ ہے، تو شہری انتظامیہ کو کون سی احتیاطی تدابیر اختیار کرنی چاہئیں؟"

# 3. Education Reform
your_query_3 = "اگر پنجاب کے سرکاری اسکولوں میں پڑھائی کا معیار 60% طلبہ کے لیے ناکافی ہے اور اساتذہ کی تربیت میں کمی اس کی وجہ ہے، تو تعلیمی بجٹ میں کس شعبے کو ترجیح دی جانی چاہیے؟"

# 4. Economic Development
your_query_4 = "اگر پاکستان میں برآمدات 25% کم ہو گئی ہیں اور بین الاقوامی معیارات پر پورا نہ اترنا اس کی بنیادی وجہ ہے، تو وزارت تجارت کو کون سی نئی پالیسیاں بنانی چاہئیں؟"

# 5. Urban Planning
your_query_5 = "اگر اسلام آباد میں ٹریفک حادثات میں 30% اضافہ ہوا ہے اور سڑکوں کی ناقص ڈیزائننگ اس کی وجہ ہے، تو سی ڈی اے کو کون سے تعمیراتی معیارات تبدیل کرنے چاہئیں؟"

# 6. Agricultural Crisis
your_query_6 = "اگر گندم کی پیداوار میں 20% کمی واقع ہوئی ہے اور کسانوں کو جدید ادویات تک رسائی نہ ہونا اس کی وجہ ہے، تو زرعی تحقیقاتی ادارے کو کون سی سہولیات فراہم کرنی چاہئیں؟"

# 7. Energy Sector
your_query_7 = "اگر لوڈ شیڈنگ میں 45% اضافہ ہوا ہے اور پرانے جنریٹرز کا ناکارہ ہونا اس کی بنیادی وجہ ہے، تو وزارت توانائی کو کون سے نئے منصوبے شروع کرنے چاہئیں؟"

# 8. Healthcare Access
your_query_8 = "اگر دیہی علاقوں میں 70% آبادی کو بنیادی صحت کی سہولیات میسر نہیں اور ڈاکٹروں کی کمی اس کی وجہ ہے، تو صوبائی حکومت کو کون سی تربیتی اسکیمیں شروع کرنی چاہئیں؟"

In [82]:
queries = [your_query_1, your_query_2, your_query_3, your_query_4, your_query_5, your_query_6, your_query_7, your_query_8]

for i, query in enumerate(queries, 1):
    print(f"\nTesting Query {i}:")
    print(query)
    result = decompose_urdu_query(query)
    print("\nDecomposition:")
    print(f"q1: {result.get('q1', 'N/A')}")
    print(f"q2: {result.get('q2', 'N/A')}")


Testing Query 1:
اگر دریائے سندھ میں آلودگی کی شرح 40% بڑھ گئی ہے اور یہ زراعت کو متاثر کر رہی ہے، تو پنجاب حکومت کو کون سے فوری اقدامات کرنے چاہئیں؟

Decomposition:
q1: دریائے سندھ میں آلودگی کی شرح 40% کی موجودہ شرح کیا گیا ہے؟
q2: زراعت کو دریائے سندھ میں آلودگی کے اثرات متاثر کرنے سے کس قسم کے اثرات مرتب ہو رہی ہیں؟

Testing Query 2:
اگر کراچی میں ڈینگی کے کیسز میں 150% اضافہ ہوا ہے اور بارشوں کا غیر معمولی نمونہ اس کی بنیادی وجہ ہے، تو شہری انتظامیہ کو کون سی احتیاطی تدابیر اختیار کرنی چاہئیں؟

Decomposition:
q1: کراچی میں ڈینگی کے کیسز میں 150% اضافہ کیا گیا ہے؟
q2: بارشوں کا غیر معمولی نمونہ اس کی بنیادی وجہ سے کیسے حالت پھاڑتی ہے شہری انتظامیہ؟

Testing Query 3:
اگر پنجاب کے سرکاری اسکولوں میں پڑھائی کا معیار 60% طلبہ کے لیے ناکافی ہے اور اساتذہ کی تربیت میں کمی اس کی وجہ ہے، تو تعلیمی بجٹ میں کس شعبے کو ترجیح دی جانی چاہیے؟

Decomposition:
q1: پنجاب کے سرکاری اسکولوں میں پڑھائی کا معیار 60% طلبہ کے لئے کیا ہے؟
q2: استعدادزدہ کی تربیت میں کمی کو کس شعبے کے وجہ سے ہے؟

Testing 

In [84]:
import ollama
ollama.pull('llama3:8b')


ProgressResponse(status='success', completed=None, total=None, digest=None)

In [86]:
import ollama

def decompose_urdu_query(urdu_query: str) -> dict:
    """Returns dictionary with q1 and q2 keys containing sub-questions"""
    refined_prompt = f"""
**Role**: You are an expert Urdu linguistic analyst specializing in question decomposition. Your task is to break down complex Urdu questions into their fundamental components.

**Task Instructions**:
1. Carefully analyze the given Urdu question to identify its core components
2. Extract exactly 2 sub-questions that:
   - Are necessary to answer the main question
   - Cover distinct aspects of the problem
   - Have clear logical progression (answer to q1 helps answer q2)
3. Both sub-questions must:
   - Be in proper Urdu language
   - Be grammatically correct
   - Be clear and concise
   - Use relevant domain terminology

**Output Format Requirements**:
- Use EXACTLY this format:
  q1: [پہلا ذیلی سوال]
  q2: [دوسرا ذیلی سوال]
- Each sub-question must be on a new line
- Do not include any additional commentary or explanation
- Do not number the questions (use only q1:/q2: prefixes)

**Example 1**:
Input: اگر لاہور میں فضائی آلودگی کی سطح دہلی سے زیادہ ہے اور فضائی آلودگی پھیپھڑوں کے کینسر کا سبب بن سکتی ہے، تو لاہور کے رہائشیوں کو کس قسم کے طبی چیک اپ کروانے چاہئیں؟
Output:
q1: لاہور اور دہلی میں فضائی آلودگی کی سطح کا موازنہ کیا ہے؟
q2: فضائی آلودگی پھیپھڑوں کے کینسر کا سبب کیسے بنتی ہے؟

**Example 2**:
Input: اگر کراچی میں بجلی کے نرخ 30% بڑھ گئے ہیں اور یہ صنعتوں کو متاثر کر رہا ہے، تو حکومت کو کون سی سبسڈیاں دینی چاہئیں؟
Output:
q1: کراچی میں بجلی کے نرخوں میں اضافے کی موجودہ شرح کیا ہے؟
q2: بجلی کے مہنگے ہونے سے صنعتوں پر کس قسم کے اثرات مرتب ہو رہے ہیں؟

**Current Task**:
Input: {urdu_query}
Output:
"""
    
    try:
        response = ollama.generate(
            model='llama3:8b',
            prompt=refined_prompt,
            options={
                'temperature': 0.5,
                'num_ctx': 2048
            }
        )
        
        output = response['response'].strip()
        
        result = {}
        for line in output.split('\n'):
            line = line.strip()
            if line.startswith('q1:'):
                result['q1'] = line[3:].strip()
            elif line.startswith('q2:'):
                result['q2'] = line[3:].strip()
        
        return result if len(result) == 2 else {}
    
    except Exception as e:
        print(f"Decomposition error: {str(e)}")
        return {}


In [87]:
queries = [your_query_1, your_query_2, your_query_3, your_query_4, your_query_5, your_query_6, your_query_7, your_query_8]

for i, query in enumerate(queries, 1):
    print(f"\nTesting Query {i}:")
    print(query)
    result = decompose_urdu_query(query)
    print("\nDecomposition:")
    print(f"q1: {result.get('q1', 'N/A')}")
    print(f"q2: {result.get('q2', 'N/A')}")


Testing Query 1:
اگر دریائے سندھ میں آلودگی کی شرح 40% بڑھ گئی ہے اور یہ زراعت کو متاثر کر رہی ہے، تو پنجاب حکومت کو کون سے فوری اقدامات کرنے چاہئیں؟

Decomposition:
q1: دریائے سندھ میں آلودگی کی شرح میں اضافے کی موجودہ شرح کیا ہے؟
q2: آلودگی سے زراعت کو متاثر کر رہی ہے، تو پنجاب حکومت کو کس قسم کے فلاحی اقدامات لینے چاہئیں؟

Testing Query 2:
اگر کراچی میں ڈینگی کے کیسز میں 150% اضافہ ہوا ہے اور بارشوں کا غیر معمولی نمونہ اس کی بنیادی وجہ ہے، تو شہری انتظامیہ کو کون سی احتیاطی تدابیر اختیار کرنی چاہئیں؟

Decomposition:
q1: کراچی میں ڈینگی کے کیسز میں اضافے کی شرح کیا ہے؟
q2: بارشوں کا غیر معمولی نمونہ ڈینگی کے اضافے کی بنیادی وجہ کیسے بنتی ہے؟

Testing Query 3:
اگر پنجاب کے سرکاری اسکولوں میں پڑھائی کا معیار 60% طلبہ کے لیے ناکافی ہے اور اساتذہ کی تربیت میں کمی اس کی وجہ ہے، تو تعلیمی بجٹ میں کس شعبے کو ترجیح دی جانی چاہیے؟

Decomposition:
q1: پنجاب کے سرکاری اسکولوں میں پڑھائی کا معیار کیا ہے؟
q2: تعلیمی بجٹ میں تربیت کی کمی کس قسم کے اثرات مرتب ہو رہے ہیں؟

Testing Query 4:
اگر پاکس

In [91]:
import ollama

def query_context_relevance_check(query_urdu: str, context_urdu: str) -> bool:
    prompt = f"""
You are an expert in understanding the meaning and relevance of questions and their supporting information.

Your task is to check whether a given *context* is relevant to a given *question*. Both the question and the context are written in Urdu.

- If the context provides information that could help answer or understand the question, reply: True
- If the context is not helpful or unrelated to the question, reply: False

---

Here is the input:

Question (Urdu): {query_urdu}

Context (Urdu): {context_urdu}

Is the context relevant to the question?
Answer (True/False):"""

    response = ollama.chat(
        model="llama3:8b",
        messages=[{"role": "user", "content": prompt}]
    )

    answer = response['message']['content'].strip().lower()
    return 'true' in answer


In [94]:
false_test_cases = [
    {
        "query": "پاکستان کا قومی پھول کون سا ہے؟",
        "context": "کراچی پاکستان کا سب سے بڑا شہر ہے۔",
        "expected": False
    },
    {
        "query": "اردو زبان کی ابتدا کہاں ہوئی؟",
        "context": "پاکستانی کرکٹ ٹیم نے ورلڈ کپ 1992 میں جیتا۔",
        "expected": False
    },
    {
        "query": "قائداعظم کی تاریخ پیدائش کیا ہے؟",
        "context": "پاکستان کا پرچم دو رنگوں پر مشتمل ہے: سبز اور سفید۔",
        "expected": False
    },
    {
        "query": "خلا میں سب سے پہلے جانے والا انسان کون تھا؟",
        "context": "پاکستان میں سب سے بلند پہاڑ کے ٹو ہے۔",
        "expected": False
    },
    {
        "query": "پاکستان کے پہلے صدر کون تھے؟",
        "context": "پاکستان میں زیادہ تر لوگ اردو اور پنجابی بولتے ہیں۔",
        "expected": False
    },
    {
        "query": "علامہ اقبال کی تعلیم کہاں سے ہوئی؟",
        "context": "کراچی میں سب سے مشہور ساحل کلفٹن ہے۔",
        "expected": False
    },
    {
        "query": "برصغیر کی تقسیم کب ہوئی؟",
        "context": "دریائے سندھ پاکستان کا سب سے بڑا دریا ہے۔",
        "expected": False
    },
    {
        "query": "لاہور کس دریا کے کنارے واقع ہے؟",
        "context": "اسلام آباد کو 1960 میں دارالحکومت بنایا گیا تھا۔",
        "expected": False
    },
    {
        "query": "پاکستان کی سب سے بڑی یونیورسٹی کون سی ہے؟",
        "context": "پاکستان میں چاروں موسم ہوتے ہیں: گرمی، سردی، خزاں، اور بہار۔",
        "expected": False
    },
    {
        "query": "بابائے قوم کا خطاب کسے دیا گیا؟",
        "context": "پاکستان کی سب سے لمبی شاہراہ قراقرم ہائی وے ہے۔",
        "expected": False
    }
]


In [95]:
for i, case in enumerate(false_test_cases, start=1):
    result = query_context_relevance_check(case["query"], case["context"])
    print(f"Test {i}: Expected: {case['expected']}, Got: {result}, {'✅' if result == case['expected'] else '❌'}")


Test 1: Expected: False, Got: False, ✅
Test 2: Expected: False, Got: False, ✅
Test 3: Expected: False, Got: False, ✅
Test 4: Expected: False, Got: False, ✅
Test 5: Expected: False, Got: False, ✅
Test 6: Expected: False, Got: False, ✅
Test 7: Expected: False, Got: False, ✅
Test 8: Expected: False, Got: False, ✅
Test 9: Expected: False, Got: False, ✅
Test 10: Expected: False, Got: False, ✅
